# Import Library

In [1]:
# Standard Imports
import numpy as np
import pandas as pd

# Additional Imports
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

# Load Credentials and Create Movie Object

In [2]:
import json
with open('/Users/ericakitano/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['API-key'])

In [3]:
# Log-in to TMDB using my API-key
tmdb.API_KEY =  login['API-key']

In [4]:
## make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

# Define `get_movie_with_rating` function

In [7]:
def get_movie_with_rating(movie_id):
    
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    info=movie.info()
    
    releases = movie.releases()
    
    #Loop through countries in releases
    for c in releases['countries']:
        #if the country is US
        if c['iso_3166_1'] == 'US':
            # save a "certification" key in "info" with the certification
            info['certification'] = c['certification']
    return info


# Test `get_movie_with_rating` function

In [8]:
# Test get_movie_with_rating function with movie_id: tt0848228 ("The Avengers")
test = get_movie_with_rating("tt0848228")
test

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 220.336,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

Verified that `info` now contains `certification` key. 